In [1]:
import pandas as pd
import numpy as np

In [2]:
adr = pd.read_excel('전국_경위도_좌표_데이터.xlsx')
adr.head()

,주소,위도,경도
0,강원도,37.860367,128.311526
1,강원도 양양군,38.075493,128.619145
2,강원도 양양군 강현면,38.149940,128.607587
3,강원도 양양군 강현면 물갑리,38.117600,128.523500
4,강원도 양양군 강현면 적은리,38.121732,128.596812


In [3]:
adr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21811 entries, 0 to 21810
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   주소      21811 non-null  object 
 1   위도      21811 non-null  float64
 2   경도      21811 non-null  float64
dtypes: float64(2), object(1)
memory usage: 511.3+ KB


In [4]:
adr.isna().sum()

주소    0
위도    0
경도    0
dtype: int64

In [6]:
adr['주소']

0                         강원도
1                     강원도 양양군
2                 강원도 양양군 강현면
3             강원도 양양군 강현면 물갑리
4             강원도 양양군 강현면 적은리
                 ...         
21806    충청북도 청주시 상당구 낭성면 이목리
21807    충청북도 청주시 상당구 낭성면 호정리
21808    충청북도 청주시 상당구 낭성면 추정리
21809    충청북도 청주시 상당구 낭성면 귀래리
21810    충청북도 청주시 상당구 낭성면 관정리
Name: 주소, Length: 21811, dtype: object

In [7]:
# '시/도' 열 생성
adr.loc[:, '시/도'] = adr['주소'].str.split().str[0]

# '시/도' 열에서 '도', '광역시', '특별시', '특별자치도'로 끝나는지 확인
adr.loc[:, '시/도'] = adr['시/도'].apply(lambda x: x if x.endswith(('도', '광역시', '특별시', '특별자치도')) else None)

# '시', '군', '구'로 끝나는 부분을 추출하여 '시/군/구' 열에 저장
def extract_sigungugu(address):
    parts = address.split()
    for part in reversed(parts):  # 끝에서부터 확인
        if part.endswith(('시', '군', '구')) and part not in ['광역시', '특별시', '특별자치시']:
            return part
    return None

adr.loc[:, '시/군/구'] = adr['주소'].apply(extract_sigungugu)

# '읍', '면', '동'으로 끝나는 부분을 추출하여 '읍/면/동' 열에 저장
def extract_eupmyeondong(address):
    parts = address.split()
    for part in reversed(parts):  # 끝에서부터 확인
        if part.endswith(('읍', '면', '동', '가')):
            return part
    return None

adr.loc[:, '읍/면/동'] = adr['주소'].apply(extract_eupmyeondong)

# '리'와 '출장'으로 끝나는 주소를 제거
adr = adr[~adr['주소'].str.endswith(('리', '출장', ')'))]

print(adr)

                     주소         위도          경도   시/도 시/군/구 읍/면/동
0                   강원도  37.860367  128.311526   강원도  None  None
1               강원도 양양군  38.075493  128.619145   강원도   양양군  None
2           강원도 양양군 강현면  38.149940  128.607587   강원도   양양군   강현면
25          강원도 양양군 현남면  37.969226  128.759488   강원도   양양군   현남면
47          강원도 양양군 현북면  38.018954  128.723115   강원도   양양군   현북면
...                 ...        ...         ...   ...   ...   ...
21722  충청북도 청주시 상당구 문의면  36.517767  127.496200  충청북도   상당구   문의면
21744  충청북도 청주시 상당구 남일면  36.586965  127.509625  충청북도   상당구   남일면
21757  충청북도 청주시 상당구 가덕면  36.553560  127.548807  충청북도   상당구   가덕면
21773  충청북도 청주시 상당구 미원면  36.636174  127.658375  충청북도   상당구   미원면
21798  충청북도 청주시 상당구 낭성면  36.622724  127.603988  충청북도   상당구   낭성면

[6756 rows x 6 columns]


In [8]:
adr.to_excel('전국_좌표데이터_최종.xlsx', index=False)